In [2]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train = pd.read_csv('../data/first_task/train.csv')
test = pd.read_csv('../data/first_task/test.csv')
sample = pd.read_csv('../data/first_task/sample_submission.csv')

In [5]:

from sklearn.model_selection import train_test_split

train_text, test_text, train_label, test_label = train_test_split(train['text'], train['sentiment'], test_size=0.2, random_state=42)


In [10]:
from transformers import AutoTokenizer

model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased-sentence', num_labels=2).to(device)
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [8]:
seq_len_train = [len(str(i).split()) for i in train['text']]
seq_len_test = [len(str(i).split()) for i in test['text']]
max_seq_len = max(max(seq_len_test), max(seq_len_train))
max_seq_len

48146

In [9]:
train_text

82633     Отель расположен очень удачно(на мой взгляд)-в...
54230     Великолепное обслуживание. Сотрудники ресепшен...
121119    Пора уже обнародовать скрываемые властями кост...
760       Коррупция в неврологическом отделении Я просто...
132950    отличный отель в современном стиле, есть небол...
                                ...                        
119879    Отмечали день рождения небольшой компании , вс...
103694    Номер был с окнами во двор, поэтому было тихо ...
131932    мы с подругой учимся рядом - потому и забегаем...
146867    Возможно, у нас бывает, что на внутренний рыно...
121958    Спасибо Прошли 2курса лечения на дневном стаци...
Name: text, Length: 151912, dtype: object

In [ ]:
train_text_array = np.array(train_text.values)
test_text_array = np.array(test_text.values)

# Преобразуйте массивы NumPy в списки
train_text_list = train_text_array.tolist()
test_text_list = test_text_array.tolist()
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)

In [3]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = Data(tokens_train, train_label)
test_dataset = Data(tokens_test, test_label)

NameError: name 'tokens_train' is not defined

In [4]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}

In [ ]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)

In [ ]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = train_dataset,
                  compute_metrics = compute_metrics)

In [ ]:
trainer.train()

In [ ]:
model_path = "fine-tune-bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()